In [ ]:
# IMPORTING LIBRARIES
import cv2
import numpy as np
import dlib
from math import hypot
import pyglet
import time
import glob
from mutagen.mp3 import MP3
import win32com.client 
from ffpyplayer.player import MediaPlayer


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

speaker = win32com.client.Dispatch("SAPI.SpVoice") 

# LOAD SOUNDS
sound = pyglet.media.load("sound.wav", streaming=False)
left_sound = pyglet.media.load("left.wav", streaming=False)
right_sound = pyglet.media.load("right.wav", streaming=False)

# FONT STYLE
font = cv2.FONT_HERSHEY_PLAIN

cap = cv2.VideoCapture(0)

# CREATING BOARD TO TAKE FOOD ORDER
board = np.zeros((1200, 800), np.uint8)
board[:] = 255

# CREATING BOARD TO WRITE TEXT
text_board = np.zeros((1200, 800), np.uint8)
text_board[:] = 255

# CREATING A BOARD
keyboard = np.zeros((870, 900, 3), np.uint8)

# MENU FOR BREAKFAST
items_set_1 = {0: "SAMBAR IDLI", 1: "IDLI VADA", 2: "ONION DOSA", 3: "MASALA DOSA", 4: "POORI CHOLE",
              5: "KESARI", 6: "PONGAL", 7: "<", 8: "ENTER"}

# MENU FOR LUNCH OR DINNER
items_set_2 = {0: "VEG THALI", 1: "LEMON RICE", 2: "CURD RICE", 3: "JEERA RICE", 4: "VEG FRIED RICE",
              5: "VEG PULAO", 6: "ROTI WITH PANEER", 7: "<", 8: "ENTER"}

# LETTERS ON LEFT SIDE OF THE KEYBOARD TO TYPE TEXT
keys_set_1 = {0: "Q", 1: "W", 2: "E", 3: "R", 4: "T",
              5: "A", 6: "S", 7: "D", 8: "F", 9: "G",
              10: "Z", 11: "X", 12: "C", 13: "V", 14: "<"}

# LETTERS ON RIGHT SIDE OF THE KEYBOARD TO TYPE TEXT
keys_set_2 = {0: "Y", 1: "U", 2: "I", 3: "O", 4: "P",
              5: "H", 6: "J", 7: "K", 8: "L", 9: "_",
              10: "B", 11: "N", 12: "M", 13: "<", 14: ">"}

# OTHER APPLICATIONS
appli = {0: "KEYBOARD", 1: "SONGS", 2: "VIDEOS", 3: "<"}

# SONGS LIST RETURNS A LIST OF ALL SONGS OF FORMAT .MP3 
songs = [i for i in glob.glob('*.{}'.format('mp3'))]

english=dict()
hindi=dict()
telugu=dict()

# THE SONGS ARE THEN DIVIDED ACCORDING TO THE LANGUAGE ENGLISH , HINDI AND TELUGU
j=len(songs)
for i in range(len(songs)):
    if songs[i].startswith('eng'):
        english[i]=songs[i]
    elif songs[i].startswith('hin'):
        hindi[i]=songs[i]
    elif songs[i].startswith('tel'):
        telugu[i]=songs[i]
        
# THE SONGS ARE THEN GIVEN AN INDEX FROM 0 TO 7 IN EACH LANGUAGE

index_list=[0,1,2,3,4,5,6,7]

english = dict(zip(index_list, list(english.values()))) 
hindi=dict(zip(index_list, list(hindi.values())))
telugu=dict(zip(index_list, list(telugu.values())))

# THE langs_set IS USE TO SELECT THE LANGUAGE FROM WHICH WE WANT A SONG.
# songs_set_1,songs_set_2,songs_set_3 ARE USED TO SELECT THE SONGS FROM ENGLISH, HINDI,TELUGU RESPECTIVELY.

langs_set = {0: "ENGLISH", 1: "HINDI", 2: "TELUGU", 3: "<"}

songs_set_1 = {0: "ALONE", 1: "BELIEVER", 2: "DESPACITO", 3: "FADED", 4: "ON MY WAY",
              5: "TAKI-TAKI", 6: "THUNDER", 7: "YOU BELONG WITH ME", 8: "<"}

songs_set_2 = {0: "ADHM TITLE TRACK", 1: "BADTAMEEZ DIL", 2: "BALAM PICHKARI", 3: "BREAKUP SONG", 4: "CHANNA MEREYA",
              5: "CONTROL SONG", 6: "KHARIYAT", 7: "WOH DIN", 8: "<"}

songs_set_3 = {0: "AAKASHAM NEE HADDU RA", 1: "ARERE AKASHAM", 2: "COLLEGE SONG", 3: "KATTUKA KANULE", 4: "MAHA THEME",
              5: "PILLA PULI", 6: "THARAGATHI GADHI", 7: "THE GUNTER SONG", 8: "<"}

# SET OF CATEGORIES
catg_set = {0: "DANCE", 1: "COMEDY", 2: "MOVIES", 3: "<"}

# SET OF DANCE VIDEOS
video_set_1 = {0: "FAKELOVE", 1: "IDOL", 2: "JungkookDance", 3: "<"}

# SET OF COMEDY VIDEOS
video_set_2 = {0: "Boss Baby 1", 1: "Boss Baby 2", 2: "Boss Baby 3", 3: "<"}

# SET OF MOVIE VIDEOS
video_set_3 = {0: "Color Photo", 1: "MCM", 2: "Miss India", 3: "<"}


# FUNCTION TO DRAW KEYBOARD OF LETTERS TO TYPE TEXT
def draw_letters(letter_index, items, letter_light):
    # Keys
    if letter_index == 0:
        x = 0
        y = 0
    elif letter_index == 1:
        x = 180
        y = 0
    elif letter_index == 2:
        x = 360
        y = 0
    elif letter_index == 3:
        x = 540
        y = 0
    elif letter_index == 4:
        x = 720
        y = 0
    elif letter_index == 5:
        x = 0
        y = 290
    elif letter_index == 6:
        x = 180
        y = 290
    elif letter_index == 7:
        x = 360
        y = 290
    elif letter_index == 8:
        x = 540
        y = 290
    elif letter_index == 9:
        x = 720
        y = 290
    elif letter_index == 10:
        x = 0
        y = 580
    elif letter_index == 11:
        x = 180
        y = 580
    elif letter_index == 12:
        x = 360
        y = 580
    elif letter_index == 13:
        x = 540
        y = 580
    elif letter_index == 14:
        x = 720
        y = 580
    width = 180
    height = 290
    th = 3 # thickness

    # items settings
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 10
    font_th = 5
    items_size = cv2.getTextSize(items, font_letter, font_scale, font_th)[0]
    width_items, height_items = items_size[0], items_size[1]
    items_x = int((width - width_items) / 2) + x
    items_y = int((height + height_items) / 2) + y

    if letter_light is True:
        cv2.rectangle(keyboard, (x + th, y + th), (x + width - th, y + height - th), (255, 255, 255), -1)
        cv2.putText(keyboard, items, (items_x, items_y), font_letter, font_scale, (51, 51, 51), font_th)
    else:
        cv2.rectangle(keyboard, (x + th, y + th), (x + width - th, y + height - th), (51, 51, 51), -1)
        cv2.putText(keyboard, items, (items_x, items_y), font_letter, font_scale, (255, 255, 255), font_th)

# FUNCTION TO DRAW A BOARD OF APPLICATIONS, LANGUAGES OF SONGS, CATEGORIES OF VIDEOS AND VIDEOS
def draw_menu(item_index, items, letter_light):
    # Keys
    if item_index == 0:
        x = 0
        y = 0
    elif item_index == 1:
        x = 450
        y = 0
    elif item_index == 2:
        x = 0
        y = 435
    elif item_index == 3:
        x = 450
        y = 435
    
    width = 450
    height = 435
    th = 3 # thickness

    # items settings
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 3.5
    font_th = 4
    items_size = cv2.getTextSize(items, font_letter, font_scale, font_th)[0]
    width_items, height_items = items_size[0], items_size[1]
    items_x = int((width - width_items) / 2) + x
    items_y = int((height + height_items) / 2) + y

    if letter_light is True:
        cv2.rectangle(keyboard, (x + th, y + th), (x + width - th, y + height - th), (255, 255, 255), -1)
        cv2.putText(keyboard, items, (items_x, items_y), font_letter, font_scale, (51, 51, 51), font_th)
    else:
        cv2.rectangle(keyboard, (x + th, y + th), (x + width - th, y + height - th), (51, 51, 51), -1)
        cv2.putText(keyboard, items, (items_x, items_y), font_letter, font_scale, (255, 255, 255), font_th)

# FUNCTION TO DRAW BOARD OF MENU OF FOOD ITEMS
def draw_items(item_index, items, letter_light):
    # Keys
    if item_index == 0:
        x = 0
        y = 0
    elif item_index == 1:
        x = 300
        y = 0
    elif item_index == 2:
        x = 600
        y = 0
    elif item_index == 3:
        x = 0
        y = 290
    elif item_index == 4:
        x = 300
        y = 290
    elif item_index == 5:
        x = 600
        y = 290
    elif item_index == 6:
        x = 0
        y = 580
    elif item_index == 7:
        x = 300
        y = 580
    elif item_index == 8:
        x = 600
        y = 580

    # items settings
    width = 300
    height = 290
    th = 3 # thickness
    
    # items settings
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 1.8
    font_th = 3
    items_size = cv2.getTextSize(items, font_letter, font_scale, font_th)[0]
    width_items, height_items = items_size[0], items_size[1]
    items_x = int((width - width_items) / 2) + x
    items_y = int((height + height_items) / 2) + y
    if letter_light is True:
        cv2.rectangle(keyboard, (x + th, y + th), (x + width - th, y + height - th), (255, 255, 255), -1)
        cv2.putText(keyboard, items, (items_x, items_y), font_letter, font_scale, (51, 51, 51), font_th)
    else:
        cv2.rectangle(keyboard, (x + th, y + th), (x + width - th, y + height - th), (51, 51, 51), -1)
        cv2.putText(keyboard, items, (items_x, items_y), font_letter, font_scale, (255, 255, 255), font_th)

# FUNCTION TO DRAW BOARD OF SONGS PLAYLIST
def draw_songs(song_index, text, letter_light):
    # Keys
    if song_index == 0:
        x = 0
        y = 0
    elif song_index == 1:
        x = 300
        y = 0
    elif song_index == 2:
        x = 600
        y = 0
    elif song_index == 3:
        x = 0
        y = 290
    elif song_index == 4:
        x = 300
        y = 290
    elif song_index == 5:
        x = 600
        y = 290
    elif song_index == 6:
        x = 0
        y = 580
    elif song_index == 7:
        x = 300
        y = 580
    elif song_index == 8:
        x = 600
        y = 580

    # Text settings
    width = 300
    height = 290
    th = 3 # thickness
    # Text settings
    font_letter = cv2.FONT_HERSHEY_PLAIN
    font_scale = 1.2
    font_th = 2
    text_size = cv2.getTextSize(text, font_letter, font_scale, font_th)[0]
    width_text, height_text = text_size[0], text_size[1]
    text_x = int((width - width_text) / 2) + x
    text_y = int((height + height_text) / 2) + y
    if letter_light is True:
        cv2.rectangle(keyboard, (x + th, y + th), (x + width - th, y + height - th), (255, 255, 255), -1)
        cv2.putText(keyboard, text, (text_x, text_y), font_letter, font_scale, (51, 51, 51), font_th)
    else:
        cv2.rectangle(keyboard, (x + th, y + th), (x + width - th, y + height - th), (51, 51, 51), -1)
        cv2.putText(keyboard, text, (text_x, text_y), font_letter, font_scale, (255, 255, 255), font_th)

# FUNCTION TO DRAW BOARD OF LEFT AND RIGHT OPTIONS
def draw_keyboard():
    rows, cols, _ = keyboard.shape
    th_lines = 4 # thickness lines
    cv2.line(keyboard, (int(cols/2) - int(th_lines/2), 0),(int(cols/2) - int(th_lines/2), rows),
             (51, 51, 51), th_lines)
    cv2.putText(keyboard, "LEFT", (80, 300), font, 6, (255, 255, 255), 5)
    cv2.putText(keyboard, "RIGHT", (80 + int(cols/2), 300), font, 6, (255, 255, 255), 5)
    
# FUNCTION TO DRAW BOARD OF BREAKFAST AND LUNCH OR DINNER OPTIONS
def draw_keyboard_1():
    rows, cols, _ = keyboard.shape
    th_lines = 4 # thickness lines
    cv2.line(keyboard, (int(cols/2) - int(th_lines/2), 0),(int(cols/2) - int(th_lines/2), rows),
             (51,51,51), th_lines)
    cv2.putText(keyboard, "BREAKFAST", (60, 350), font, 4, (255, 255, 255), 5)
    cv2.putText(keyboard, "LUNCH", (80 + int(cols/2), 200), font, 4, (255, 255, 255), 5)
    cv2.putText(keyboard, "OR", (80 + int(cols/2), 350), font, 4, (255, 255, 255), 5)
    cv2.putText(keyboard, "DINNER", (80 + int(cols/2), 500), font,4, (255, 255, 255), 5)

# FUNCTION TO DRAW BOARD OF FOOD AND OTHER APPLICATIONS OPTIONS 
def draw_keyboard_2():
    rows, cols, _ = keyboard.shape
    th_lines = 4 # thickness lines
    cv2.line(keyboard, (int(cols/2) - int(th_lines/2), 0),(int(cols/2) - int(th_lines/2), rows),
             (51, 51, 51), th_lines)
    cv2.putText(keyboard, "FOOD", (80, 350), font, 3.5, (255, 255, 255), 5)
    cv2.putText(keyboard, "OTHER", (65 + int(cols/2), 300), font, 3, (255, 255, 255), 5)
    cv2.putText(keyboard, "APPLICATIONS", (65 + int(cols/2), 400), font, 3, (255, 255, 255), 5)

# FUNCTION TO DRAW BOARD OF NO AND YES OPTIONS
def draw_keyboard_3():
    rows, cols, _ = keyboard.shape
    th_lines = 4 # thickness lines
    cv2.line(keyboard, (int(cols/2) - int(th_lines/2), 0),(int(cols/2) - int(th_lines/2), rows),
             (51, 51, 51), th_lines)
    cv2.putText(keyboard, "NO", (80, 300), font, 6, (255, 255, 255), 5)
    cv2.putText(keyboard, "YES", (80 + int(cols/2), 300), font, 6, (255, 255, 255), 5)

# FUNCTION TO FIND MID POINT USED IN CALCULATION OF GAZE RATIO
def midpoint(p1 ,p2):
    return int((p1.x + p2.x)/2), int((p1.y + p2.y)/2)

# FUNCTION TO FIND BLINKING RATIO OF EYES
def get_blinking_ratio(eye_points, facial_landmarks):
    left_point = (facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y)
    right_point = (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y)
    center_top = midpoint(facial_landmarks.part(eye_points[1]), facial_landmarks.part(eye_points[2]))
    center_bottom = midpoint(facial_landmarks.part(eye_points[5]), facial_landmarks.part(eye_points[4]))

    hor_line_lenght = hypot((left_point[0] - right_point[0]), (left_point[1] - right_point[1]))
    ver_line_lenght = hypot((center_top[0] - center_bottom[0]), (center_top[1] - center_bottom[1]))

    ratio = hor_line_lenght / ver_line_lenght
    return ratio

# FUNCTION TO GET CONTOUR POINTS OF EYE
def eyes_contour_points(facial_landmarks):
    left_eye = []
    right_eye = []
    for n in range(36, 42):
        x = facial_landmarks.part(n).x
        y = facial_landmarks.part(n).y
        left_eye.append([x, y])
    for n in range(42, 48):
        x = facial_landmarks.part(n).x
        y = facial_landmarks.part(n).y
        right_eye.append([x, y])
    left_eye = np.array(left_eye, np.int32)
    right_eye = np.array(right_eye, np.int32)
    return left_eye, right_eye

# FUNCTION TO FIND GAZE RATIO OF EYES
def get_gaze_ratio(eye_points, facial_landmarks):
    eye_region = np.array([(facial_landmarks.part(eye_points[0]).x, facial_landmarks.part(eye_points[0]).y),
                                (facial_landmarks.part(eye_points[1]).x, facial_landmarks.part(eye_points[1]).y),
                                (facial_landmarks.part(eye_points[2]).x, facial_landmarks.part(eye_points[2]).y),
                                (facial_landmarks.part(eye_points[3]).x, facial_landmarks.part(eye_points[3]).y),
                                (facial_landmarks.part(eye_points[4]).x, facial_landmarks.part(eye_points[4]).y),
                                (facial_landmarks.part(eye_points[5]).x, facial_landmarks.part(eye_points[5]).y)], np.int32)
    #cv2.polylines(frame, [eye_region], True, (0,0,255), 2)
    #print(eye_region)
    
    min_x = np.min(eye_region[:, 0])
    max_x = np.max(eye_region[:, 0])
    min_y = np.min(eye_region[:, 1])
    max_y = np.max(eye_region[:, 1])
    
    height, width, _ = frame.shape
    mask = np.zeros((height, width), np.uint8)
    cv2.polylines(mask, [eye_region], True, 255, 2)
    cv2.fillPoly(mask, [eye_region], 255)
    eye = cv2.bitwise_and(gray, gray, mask=mask)
        
        
    #eye=frame[min_y: max_y, min_x: max_x]
    #gray_eye=cv2.cvtColor(eye, cv2.COLOR_BGR2GRAY)
    gray_eye = eye[min_y: max_y, min_x: max_x]
    _, threshold_eye = cv2.threshold(gray_eye, 70, 255, cv2.THRESH_BINARY)
    
    threshold_eye = cv2.resize(threshold_eye, None, fx=5, fy=5)
    eye = cv2.resize(gray_eye, None, fx=5, fy=5)
    cv2.imshow("Eye", eye)
    cv2.imshow("Threshold", threshold_eye)
        
    height, width=threshold_eye.shape
    left_side_threshold=threshold_eye[0:height,0:int(width/2)]
    left_side_white=cv2.countNonZero(left_side_threshold)
        
    right_side_threshold=threshold_eye[0:height,int(width/2):width]
    right_side_white=cv2.countNonZero(right_side_threshold)
    
    #cv2.imshow("Left",left_side_threshold)
    #cv2.imshow("Right",right_side_threshold)
    #cv2.putText(frame, str(left_side_white), (50,0), font, 2, (0,0,255), 3)
    #cv2.putText(frame, str(right_side_white), (50,150), font, 2, (0,0,255), 3)
    
    if left_side_white == 0:
        gaze_ratio=1
    elif right_side_white == 0:
        gaze_ratio=5
    else:
        gaze_ratio=left_side_white/right_side_white
    
    
    return gaze_ratio
    
# COUNTERS
frames = 0
item_index = 0
letter_index = 0
lang_index = 0
song_index = 0
appli_index=0
video_index = 0
category_index = 0
blinking_frames = 0
frames_to_blink = 6
frames_active_letter = 15
order=0
length=80
selected_frame = 0
keyboard_selection_frames = 0

# INTIALIZING REQUIRED VARIABLES
text=""
items = []
keyboard_selected = "FOOD"
last_keyboard_selected = "FOOD"
keyboard_selected_1 = "BREAKFAST"
last_keyboard_selected_1 = "BREAKFAST"
keyboard_selected_2 = "LEFT"
last_keyboard_selected_2 = "LEFT"
keyboard_selected_3 = "NO"
last_keyboard_selected_3 = "NO"


while True:
    _, frame = cap.read()
    
    keyboard[:] = (26,26,26)
    
    frames += 1
    
    # CONVERTING COLOURED FRAME TO GRAY FOR MORE ACCURACY
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # GETTING SHAPE OF THE FRAME
    rows, cols, _ = frame.shape
    
    # DRAWING WHITE SPACE FOR LOADING BAR
    frame[rows - 50: rows, 0: cols] = (255, 255, 255)
    if selected_frame==0:
        draw_keyboard_2()
    
    """ FRAMES AND THEIR ASSIGNED NUMBERS
        0 - > FOOD, OTHER APPLICATIONS
        1 - > BREAKFAST, LUNCH OR DINNER
        2 - > BREAKFAST MENU
        3 - > LUNCH OR DINNER MENU
        4 - > OTHER APPLICATION
        5 - > LEFT, RIGHT
        6 - > LEFT SIDE OF THE KEYBOARD
        7 - > RIGHT SIDE OF THE KEYBOARD
        8 - > NO, YES
        9 - > LANG_SELECTION
        10- > ENGLISH
        11- > HINDI
        12- > TELUGU
        13- > CATEGORIES
        14- > DANCE VIDEOS
        15- > COMEDY VIDEOS
        16- > MOVIE VIDEOS"""
    
    # ASSIGNING ITEMS_SET WITH RESPECTIVE MENUS BASED KEYBOARD SELECTED
    if keyboard_selected_1 == "BREAKFAST":
        items_set = items_set_1
    elif keyboard_selected_1 == "LUNCH OR DINNER":
        items_set = items_set_2
    
    # INITIALIZING ACTIVE LETTER BASED ON KEYBOARD SELECTED
    if selected_frame==6:
        active_letter=keys_set_1[letter_index]
    elif selected_frame==7:
        active_letter=keys_set_2[letter_index]
    elif selected_frame==2 or selected_frame==3:
        active_letter=items_set[item_index]
    elif selected_frame==4:
        active_letter=appli[appli_index]
    elif selected_frame==9:
        active_letter=langs_set[lang_index]
    elif selected_frame==10:
        active_letter=songs_set_1[song_index]
    elif selected_frame==11:
        active_letter=songs_set_2[song_index]
    elif selected_frame==12:
        active_letter=songs_set_3[song_index]
    elif selected_frame==13:
        active_letter=catg_set[category_index]
    elif selected_frame==14:
        active_letter=video_set_1[video_index]
    elif selected_frame==15:
        active_letter=video_set_2[video_index]
    elif selected_frame==16:
        active_letter=video_set_3[video_index]
        
    # DETECTING FACE
    faces = detector(gray)
    
    for face in faces:
        
        # DRAWING KEYBOARDS BASED ON THE selected_frame
        if selected_frame==0:
            draw_keyboard_2()
        elif selected_frame==1:
            draw_keyboard_1()
        elif selected_frame==5:
            draw_keyboard()
        elif selected_frame==8:
            draw_keyboard_3()
            
        # GETTING FACIAL LANDMARKS USING PREDICTOR
        landmarks = predictor(gray, face)
        
        # GETTING CONTOUR POINTS USING EYE_CONTOUR_POINTS() FUNCTION
        left_eye, right_eye = eyes_contour_points(landmarks)

        # DETECTING BLINKING RATIO
        left_eye_ratio = get_blinking_ratio([36, 37, 38, 39, 40, 41], landmarks)
        right_eye_ratio = get_blinking_ratio([42, 43, 44, 45, 46, 47], landmarks)
        blinking_ratio = (left_eye_ratio + right_eye_ratio) / 2
        
        
        # DRAWING LINES ON EYE 
        cv2.polylines(frame, [left_eye], True, (0, 0, 255), 2)
        cv2.polylines(frame, [right_eye], True, (0, 0, 255), 2)
        
        # FOOD, OTHER APPLICATIONS
        if selected_frame==0:
            time.sleep(0.01)
            # DETECTING GAZE TO SELECT LEFT OR RIGHT KEYBOARD
            gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
            gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
            gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye) / 2
            
            # IF GAZE_RATIO IS LESS THAN 0.9 THEN THE PERSON IS LOOKING RIGHT 
            # OPTION ON THE RIGHT IS SELCTED AND APPROPRIATE ACTION IS PERFORMED
            if gaze_ratio < 0.9:
                keyboard_selected = "OTHER APPLICATIONS"
                keyboard_selection_frames += 1
                # IF GAZE REMAINS SAME FOR MORE THAN 15 FRAMES THEN MOVE TO THE KEYBOARD
                if keyboard_selection_frames == 15:
                    selected_frame=4
                    t="other applications"
                    speaker.Speak(t)
                    # SET FRAMES COUNT TO 0 AFTER KEYBOARD IS SELECTED 
                    frames = 0
                    keyboard_selection_frames = 0
                if keyboard_selected != last_keyboard_selected:
                    last_keyboard_selected = keyboard_selected
                    keyboard_selection_frames = 0 
                    
            # IF GAZE_RATIO IS GREATER THAN 0.9 THEN THE PERSON IS LOOKING LEFT 
            # OPTION ON THE LEFT IS SELCTED AND APPROPRIATE ACTION IS PERFORMED
            else:
                keyboard_selected = "FOOD"
                keyboard_selection_frames += 1
                # IF GAZE REMAINS SAME FOR MORE THAN 15 FRAMES THEN MOVE TO THE KEYBOARD
                if keyboard_selection_frames == 15:
                    selected_frame=1
                    t="food"
                    speaker.Speak(t)
                    # SET FRAMES COUNT TO 0 AFTER KEYBOARD IS SELECTED
                    frames = 0
                if keyboard_selected != last_keyboard_selected:
                    last_keyboard_selected = keyboard_selected
                    keyboard_selection_frames = 0
        
        # BREAKFAST, LUNCH OR DINNER
        elif selected_frame==1:
            # DETECTING GAZE TO SELECT LEFT OR RIGHT KEYBOARD
            gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
            gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
            gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye) / 2
            
            # IF GAZE_RATIO IS LESS THAN 0.9 THEN THE PERSON IS LOOKING RIGHT 
            # OPTION ON THE RIGHT IS SELCTED AND APPROPRIATE ACTION IS PERFORMED
            if gaze_ratio < 0.9:
                keyboard_selected_1 = "LUNCH OR DINNER"
                order=3
                keyboard_selection_frames += 1
                # IF GAZE REMAINS SAME FOR MORE THAN 15 FRAMES THEN MOVE TO THE KEYBOARD
                if keyboard_selection_frames == 15:
                    selected_frame=3
                    t="Lunch or dinner"
                    speaker.Speak(t)
                    # SET FRAMES COUNT TO 0 AFTER KEYBOARD IS SELECTED 
                    frames = 0
                    keyboard_selection_frames = 0
                if keyboard_selected_1 != last_keyboard_selected_1:
                    last_keyboard_selected_1 = keyboard_selected_1
                    keyboard_selection_frames = 0
                    
            # IF GAZE_RATIO IS GREATER THAN 0.9 THEN THE PERSON IS LOOKING LEFT 
            # OPTION ON THE LEFT IS SELCTED AND APPROPRIATE ACTION IS PERFORMED    
            else:
                keyboard_selected_1 = "BREAKFAST"
                order=2
                keyboard_selection_frames += 1
                # IF GAZE REMAINS SAME FOR MORE THAN 15 FRAMES THEN MOVE TO THE KEYBOARD
                if keyboard_selection_frames == 15:
                    selected_frame=2
                    t="breakfast"
                    speaker.Speak(t)
                    # SET FRAMES COUNT TO 0 AFTER KEYBOARD IS SELECTED 
                    frames = 0
                if keyboard_selected_1 != last_keyboard_selected_1:
                    last_keyboard_selected_1 = keyboard_selected_1
                    keyboard_selection_frames = 0
        
        # BREAKFAST MENU
        elif selected_frame==2:
            if blinking_ratio > 5:
                
                blinking_frames += 1
                frames -= 1

                # SHOW GREEN EYES WHEN CLOSED
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter != "<" and active_letter != "ENTER":
                        items.append(active_letter)
                        sound.play()
                        selected_frame = 2
                    elif active_letter == "ENTER":
                        if len(items)==0:
                            t="Kindly place your order   "
                            speaker.Speak(t)
                            selected_frame=2
                        else:
                            order=2
                            t="Your order is    "
                            speaker.Speak(t)
                            for i in items:
                                speaker.Speak(i)
                            selected_frame=8
                    else:
                        items.clear()
                        board[:] = 255
                        cv2.imshow("Board", board)
                        selected_frame=0
                    

            else:
                blinking_frames = 0
        
        # LUNCH OR DINNER MENU
        elif selected_frame==3:
            if blinking_ratio > 5:
                
                blinking_frames += 1
                frames -= 1

                # SHOW GREEN EYES WHEN CLOSED
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter != "<" and active_letter != "ENTER":
                        items.append(active_letter)
                        sound.play()
                        selected_frame = 3
                    elif active_letter == "ENTER":
                        if len(items)==0:
                            t="Kindly place your order   "
                            speaker.Speak(t)
                            selected_frame=3
                        else:
                            order=3
                            t="Your order is    "
                            speaker.Speak(t)
                            for i in items:
                                speaker.Speak(i)
                            selected_frame=8
                    else:
                        items.clear()
                        board[:] = 255
                        cv2.imshow("Board", board)
                        selected_frame=0
                    

            else:
                blinking_frames = 0
        
        # OTHER APPLICATIONS
        elif selected_frame==4:
            if blinking_ratio > 5:
                
                blinking_frames += 1
                frames -= 1

                # SHOW GREEN EYES WHEN CLOSED
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter == "KEYBOARD":
                        sound.play()
                        selected_frame = 5
                    elif active_letter == "SONGS":
                        sound.play()
                        selected_frame = 9
                    elif active_letter == "VIDEOS":
                        sound.play()
                        selected_frame = 13
                    else:
                        selected_frame=0
                
            else:
                blinking_frames = 0
        
        # LEFT, RIGHT
        elif selected_frame==5:
            # DETECTING GAZE TO SELECT LEFT OR RIGHT KEYBOARD
            gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
            gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
            gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye) / 2
            
            # IF GAZE_RATIO IS LESS THAN 0.9 THEN THE PERSON IS LOOKING RIGHT 
            # OPTION ON THE RIGHT IS SELCTED AND APPROPRIATE ACTION IS PERFORMED
            if gaze_ratio < 0.9:
                keyboard_selected_2= "RIGHT"
                keyboard_selection_frames += 1
                # IF GAZE REMAINS SAME FOR MORE THAN 15 FRAMES THEN MOVE TO THE KEYBOARD
                if keyboard_selection_frames == 15:
                    selected_frame=7
                    right_sound.play()
                    # SET FRAMES COUNT TO 0 AFTER KEYBOARD IS SELECTED 
                    frames = 0
                    keyboard_selection_frames = 0
                if keyboard_selected_2 != last_keyboard_selected_2:
                    last_keyboard_selected_2 = keyboard_selected_2
                    keyboard_selection_frames = 0
            # IF GAZE_RATIO IS GREATER THAN 0.9 THEN THE PERSON IS LOOKING LEFT
            # OPTION ON THE LEFT IS SELCTED AND APPROPRIATE ACTION IS PERFORMED
            else:
                keyboard_selected_2 = "LEFT"
                keyboard_selection_frames += 1
                # IF GAZE REMAINS SAME FOR MORE THAN 15 FRAMES THEN MOVE TO THE KEYBOARD
                if keyboard_selection_frames == 15:
                    selected_frame=6
                    left_sound.play()
                    # SET FRAMES COUNT TO 0 AFTER KEYBOARD IS SELECTED 
                    frames = 0
                if keyboard_selected_2 != last_keyboard_selected_2:
                    last_keyboard_selected_2 = keyboard_selected_2
                    keyboard_selection_frames = 0       
        
        # LEFT SIDE OF THE KEYBOARD
        elif selected_frame==6:
            if blinking_ratio > 5:
                
                blinking_frames += 1
                frames -= 1

                # SHOW GREEN EYES WHEN CLOSED
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter != "<":
                        text+=active_letter
                        sound.play()
                        selected_frame = 6
                    else:
                        selected_frame=5
                   

            else:
                blinking_frames = 0
        
        # RIGHT SIDE OF THE KEYBOARD
        elif selected_frame==7:
            if blinking_ratio > 5:
                
                blinking_frames += 1
                frames -= 1

                # SHOW GREEN EYES WHEN CLOSED
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter != "<" and active_letter!="_" and active_letter!=">":
                        text+=active_letter
                        sound.play()
                        selected_frame = 7
                    elif active_letter=="_":
                        text+=" "
                        sound.play()
                        selected_frame = 7
                    elif active_letter == ">": 
                        selected_frame = 8
                    else:
                        selected_frame=5

            else:
                blinking_frames = 0
        
        # NO, YES
        elif selected_frame==8:
            # DETECTING GAZE TO SELECT LEFT OR RIGHT KEYBOARD
            gaze_ratio_left_eye = get_gaze_ratio([36, 37, 38, 39, 40, 41], landmarks)
            gaze_ratio_right_eye = get_gaze_ratio([42, 43, 44, 45, 46, 47], landmarks)
            gaze_ratio = (gaze_ratio_right_eye + gaze_ratio_left_eye) / 2
            
            # IF GAZE_RATIO IS LESS THAN 0.9 THEN THE PERSON IS LOOKING RIGHT 
            # OPTION ON THE RIGHT IS SELCTED AND APPROPRIATE ACTION IS PERFORMED
            if gaze_ratio < 0.9:
                keyboard_selected_3= "YES"
                keyboard_selection_frames += 1
                # IF GAZE REMAINS SAME FOR MORE THAN 15 FRAMES THEN MOVE TO THE KEYBOARD
                if keyboard_selection_frames == 15:
                    if order==2 or order==3:
                        items.clear()
                        t="Thank you for ordering.   Your food will be delivered soon"
                        order=0
                        speaker.Speak(t)  
                        selected_frame=0
                        board[:] = 255
                        cv2.imshow("Board", board)
                    else:
                        speaker.Speak(text)
                        text=""
                        items.clear()
                        text_board[:] = 255
                        board[:] = 255
                        cv2.imshow("Board", text_board)
                        selected_frame=0
                    # SET FRAMES COUNT TO 0 AFTER KEYBOARD IS SELECTED 
                    frames = 0
                    keyboard_selection_frames = 0
                if keyboard_selected_3 != last_keyboard_selected_3:
                    last_keyboard_selected_3 = keyboard_selected_3
                    keyboard_selection_frames = 0
            # IF GAZE_RATIO IS GREATER THAN 0.9 THEN THE PERSON IS LOOKING LEFT 
            # OPTION ON THE LEFT IS SELCTED AND APPROPRIATE ACTION IS PERFORMED
            else:
                keyboard_selected_3 = "NO"
                keyboard_selection_frames += 1
                # IF GAZE REMAINS SAME FOR MORE THAN 15 FRAMES THEN MOVE TO THE KEYBOARD
                if keyboard_selection_frames == 15:                   
                    if order==2 or order==3:
                        items.clear()
                        t="Kindly replace your order"
                        order=0
                        speaker.Speak(t)
                        board[:] = 255
                        cv2.imshow("Board", board)
                        selected_frame=order
                    else:
                        t="Kindly type the text again"
                        speaker.Speak(t)
                        selected_frame=5
                        sound.play()
                        text_board[:] = 255
                        cv2.imshow("Board", text_board)
                        text=""
                        items.clear()
                    # SET FRAMES COUNT TO 0 AFTER KEYBOARD IS SELECTED 
                    frames = 0
                if keyboard_selected_3 != last_keyboard_selected_3:
                    last_keyboard_selected_3 = keyboard_selected_3
                    keyboard_selection_frames = 0       
        
        # LANG_SELECTION
        elif selected_frame==9:
            if blinking_ratio > 5:
                blinking_frames += 1
                frames -= 1
                
                # SHOW GREEN EYES WHEN CLOSED
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter == "ENGLISH":
                        t="English songs"
                        speaker.Speak(t)
                        selected_frame = 10
                    elif active_letter == "HINDI":
                        t="Hindi songs"
                        speaker.Speak(t)
                        selected_frame = 11
                    elif active_letter == "TELUGU":
                        t="Telugu songs"
                        speaker.Speak(t)
                        selected_frame = 12
                    else:
                        sound.play()
                        selected_frame = 4                   
            else:
                blinking_frames = 0
        
        # ENGLISH SONGS
        elif selected_frame==10:
            if blinking_ratio > 5:
                blinking_frames += 1
                frames -= 1
                
                # SHOW GREEN EYES WHEN CLOSED
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter != "<":
                        t="Playing "+active_letter
                        speaker.Speak(t)
                        eng_song_name="eng_"+active_letter+".mp3"
                        eng_song = pyglet.media.load(eng_song_name, streaming=False)
                        audio = MP3(eng_song_name)
                        sound.play()
                        eng_song.play()
                        time.sleep(audio.info.length)
                        selected_frame = 10
                    else:
                        selected_frame = 9
                        sound.play()
            else:
                blinking_frames = 0   
        
        # HINDI SONGS
        elif selected_frame==11:           
            if blinking_ratio > 5:
                blinking_frames += 1
                frames -= 1
                
                # SHOW GREEN EYES WHEN CLOSED
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter != "<":
                        t="Playing "+active_letter
                        speaker.Speak(t)
                        hin_song_name="hin_"+active_letter+".mp3"
                        hin_song = pyglet.media.load(hin_song_name, streaming=False)
                        audio = MP3(hin_song_name)
                        sound.play()
                        hin_song.play()
                        time.sleep(audio.info.length)
                        selected_frame = 11
                    else:
                        selected_frame = 9
                        sound.play()                   
            else:
                blinking_frames = 0
        
        # TELUGU SONGS
        elif selected_frame==12:
            if blinking_ratio > 5:
                blinking_frames += 1
                frames -= 1
                
                # SHOW GREEN EYES WHEN CLOSED
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter != "<":
                        t="Playing "+active_letter
                        speaker.Speak(t)
                        tel_song_name="tel_"+active_letter+".mp3"
                        tel_song = pyglet.media.load(tel_song_name, streaming=False)
                        audio = MP3(tel_song_name)
                        sound.play()
                        tel_song.play()
                        time.sleep(audio.info.length)
                        selected_frame = 12
                    else:
                        selected_frame = 9
                        sound.play()
            else:
                blinking_frames = 0 
    
        # VIDEO CATEGORY FRAME
        elif selected_frame==13:
            if blinking_ratio > 5:
                blinking_frames += 1
                frames -= 1

                # Show green eyes when closed
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                if blinking_frames == frames_to_blink:
                    if active_letter != "<":
                        if active_letter == "DANCE":
                            sound.play()
                            selected_frame = 14
                        elif active_letter == "COMEDY":
                            sound.play()
                            selected_frame = 15
                        elif active_letter == "MOVIES":
                            sound.play()
                            selected_frame = 16
                    else:
                        selected_frame=0
            else:
                blinking_frames = 0
        
        # DANCE VIDEOS 
        elif selected_frame==14:
            if blinking_ratio > 5:
                blinking_frames += 1
                frames -= 1

                # Show green eyes when closed
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                # Typing letter
                if blinking_frames == frames_to_blink:
                    if active_letter != "<":
                        dance_video_name="dance_"+active_letter+".mp4"
                        sound.play()
                        cap1 = cv2.VideoCapture(dance_video_name)
                        player1 = MediaPlayer(dance_video_name)
                        while(cap1.isOpened()):
                            cv2.namedWindow("Video",cv2.WINDOW_AUTOSIZE)
                            ret1, frame1 = cap1.read()
                            audio_frame1, val1 = player1.get_frame()
                            if ret1 == False:
                                break
                            if val1 != 'eof' and audio_frame1 is not None:
                                img1, t1 = audio_frame1
                            cv2.imshow("Video", frame1)
                            cv2.waitKey(18)
                            if cv2.getWindowProperty("Video",cv2.WND_PROP_VISIBLE)<1:
                                break
                                
                        player1.close_player()        
                        cap1.release()
                        cv2.destroyWindow("Video")
                        selected_frame = 14
                    else:
                        sound.play()
                        selected_frame = 13

            else:
                blinking_frames = 0   
        
        # COMEDY VIDEOS
        elif selected_frame==15:
            # Detect the blinking to select the key that is lighting up
            if blinking_ratio > 5.4:
                # cv2.putText(frame, "BLINKING", (50, 150), font, 4, (255, 0, 0), thickness=3)
                blinking_frames += 1
                frames -= 1

                # Show green eyes when closed
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                # Typing letter
                if blinking_frames == frames_to_blink:
                    if active_letter != "<":
                        comedy_video_name="comedy_"+active_letter+".mp4"
                        sound.play()
                        cap2 = cv2.VideoCapture(comedy_video_name)
                        player2 = MediaPlayer(comedy_video_name)
                        while(cap2.isOpened()):
                            cv2.namedWindow("Video",cv2.WINDOW_AUTOSIZE)
                            ret2, frame2 = cap2.read()
                            audio_frame2, val2 = player2.get_frame()
                            if ret2 == False:
                                break
                            if val2 != 'eof' and audio_frame2 is not None:
                                img2, t2 = audio_frame2
                                if cv2.getWindowProperty("Video",cv2.WND_PROP_VISIBLE)<1:
                                    break
                            cv2.imshow("Video", frame2)
                            cv2.waitKey(26)
                            if cv2.getWindowProperty("Video",cv2.WND_PROP_VISIBLE)<1:
                                break
                            
                        player2.close_player()
                        cap2.release()
                        cv2.destroyWindow("Video")
                        selected_frame = 15
                    else:
                        sound.play()
                        selected_frame = 13

            else:
                blinking_frames = 0
        
        # MOVIE VIDEOS
        elif selected_frame==16:
            # Detect the blinking to select the key that is lighting up
            if blinking_ratio > 5.4:
                # cv2.putText(frame, "BLINKING", (50, 150), font, 4, (255, 0, 0), thickness=3)
                blinking_frames += 1
                frames -= 1

                # Show green eyes when closed
                cv2.polylines(frame, [left_eye], True, (0, 255, 0), 2)
                cv2.polylines(frame, [right_eye], True, (0, 255, 0), 2)

                # Typing letter
                if blinking_frames == frames_to_blink:
                    if active_letter != "<":
                        movie_video_name="Movies_"+active_letter+".mp4"
                        sound.play()
                        cap3 = cv2.VideoCapture(movie_video_name)
                        player3 = MediaPlayer(movie_video_name)
                        while(cap3.isOpened()):
                            cv2.namedWindow("Video",cv2.WINDOW_AUTOSIZE)
                            ret3, frame3 = cap3.read()
                            audio_frame3, val3 = player3.get_frame()
                            if ret3 == False:
                                break
                            if val3 != 'eof' and audio_frame3 is not None:
                                img3, t3 = audio_frame3
                            cv2.imshow("Video", frame3)
                            cv2.waitKey(27)
                            if cv2.getWindowProperty("Video",cv2.WND_PROP_VISIBLE)<1:
                                break
                            
                        player3.close_player()
                        cap3.release()
                        cv2.destroyWindow("Video")
                        selected_frame = 16
                    else:
                        sound.play()
                        selected_frame = 13

            else:
                blinking_frames = 0

    # DRAW THE BREAKFAST MENU
    if selected_frame == 2:
        if frames == frames_active_letter:
            item_index += 1
            frames = 0
        if item_index == 9:
            item_index = 0
        for i in range(9):
            if i == item_index:
                light = True
            else:
                light = False
            draw_items(i, items_set[i], light)
            
    # DRAW THE LUNCH OR DINNER MENU
    elif selected_frame == 3:
        if frames == frames_active_letter:
            item_index += 1
            frames = 0
        if item_index == 9: 
            item_index = 0
        for i in range(9):
            if i == item_index:
                light = True
            else:
                light = False
            draw_items(i, items_set[i], light)
    
    # DRAW THE OTHER APPLICATIONS
    elif selected_frame == 4:
        if frames == frames_active_letter:
            appli_index += 1
            frames = 0
        if appli_index == 4:
            appli_index = 0
        for i in range(4):
            if i == appli_index:
                light = True
            else:
                light = False
            draw_menu(i, appli[i], light)
            
    # DRAW THE LEFT SIDE OF THE KEYBOARD
    elif selected_frame == 6:
        if frames == frames_active_letter:
            letter_index += 1
            frames = 0
        if letter_index == 15:
            letter_index = 0
        for i in range(15):
            if i == letter_index:
                light = True
            else:
                light = False
            draw_letters(i, keys_set_1[i], light)
            
    # DRAW THE RIGHT SIDE OF THE KEYBOARD
    elif selected_frame == 7:
        if frames == frames_active_letter:
            letter_index += 1
            frames = 0
        if letter_index == 15:
            letter_index = 0
        for i in range(15):
            if i == letter_index:
                light = True
            else:
                light = False
            draw_letters(i, keys_set_2[i], light)
            
    # DRAW THE LANGUAGE SELECTION MENU
    elif selected_frame == 9:
        if frames == frames_active_letter:
            lang_index += 1
            frames = 0
        if lang_index == 4:
            lang_index = 0
        for i in range(4):
            if i == lang_index:
                light = True
            else:
                light = False
            draw_menu(i, langs_set[i], light)
            
    # DRAW THE ENGLISH SONGS MENU
    elif selected_frame == 10:
        if frames == frames_active_letter:
            song_index += 1
            frames = 0
        if song_index == 9:
            song_index = 0
        for i in range(9):
            if i == song_index:
                light = True
            else:
                light = False
            draw_songs(i, songs_set_1[i], light)
                
    # DRAW THE HINDI SONGS MENU
    elif selected_frame == 11:
        if frames == frames_active_letter:
            song_index += 1
            frames = 0
        if song_index == 9:
            song_index = 0
        for i in range(9):
            if i == song_index:
                light = True
            else:
                light = False
            draw_songs(i,songs_set_2[i], light)
                
    # DRAW THE TELUGU SONGS MENU        
    elif selected_frame == 12:
        if frames == frames_active_letter:
            song_index += 1
            frames = 0
        if song_index == 9:
            song_index = 0
        for i in range(9):
            if i == song_index:
                light = True
            else:
                light = False
            draw_songs(i, songs_set_3[i], light)
    
    # DRAW CATEGORY KEYBOARD
    if selected_frame ==13:
        if frames == frames_active_letter:
            category_index += 1
            frames = 0
        if category_index == 4:
            category_index = 0
        for i in range(4):
            if i == category_index:
                light = True
            else:
                light = False
            draw_menu(i, catg_set[i], light)
            
    # DRAW DANCE VIDEOS KEYBOARD
    elif selected_frame ==14:
        if frames == frames_active_letter:
            video_index += 1
            frames = 0
        if video_index == 4:
            video_index = 0
        for i in range(4):
            if i == video_index:
                light = True
            else:
                light = False
            draw_menu(i, video_set_1[i], light)
            
    # DRAW COMEDY VIDEOS KEYBOARD
    elif selected_frame ==15:
        if frames == frames_active_letter:
            video_index += 1
            frames = 0
        if video_index == 4:
            video_index = 0
        for i in range(4):
            if i == video_index:
                light = True
            else:
                light = False
            draw_menu(i, video_set_2[i], light)
            
    # DRAW MOVIE VIDEOS KEYBOARD
    elif selected_frame ==16:
        if frames == frames_active_letter:
            video_index += 1
            frames = 0
        if video_index == 4:
            video_index = 0
        for i in range(4):
            if i == video_index:
                light = True
            else:
                light = False
            draw_menu(i, video_set_3[i], light)
                
    # WRITING TEXT ON THE BOARD
    
    if selected_frame==6 or selected_frame==7:
        cv2.putText(text_board, text, (80, 100), font, 5, 0, 3)
        
    # WRITING ORDER TAKEN ON THE BOARD
    
    elif selected_frame==2 or selected_frame==3:
        length=100
        if len(items)!=0:
            for i in items:
                cv2.putText(board, i, (80, length), font, 3, 0, 3)
                length=length+50
    
    # LOADING BAR
    percentage_blinking = blinking_frames / frames_to_blink
    loading_x = int(cols * percentage_blinking)

    #cv2.imshow("Frame", frame)
    cv2.imshow("Virtual keyboard", keyboard)
    if selected_frame==2 or selected_frame==3:
        cv2.imshow("Board", board)
    elif selected_frame==6 or selected_frame==7:
        cv2.imshow("Board", text_board)

    key = cv2.waitKey(1)
    if key == 27:
        break
    

cap.release()
cv2.destroyAllWindows()